# Downloading data and unzipping

In [ ]:
# !gdown https://drive.google.com/uc?id=1STTlixkDCgeUMgWqrrYmiwaa3w6B9MFe&export=download

Downloading...
From: https://drive.google.com/uc?id=1STTlixkDCgeUMgWqrrYmiwaa3w6B9MFe
To: /content/assignment_1_data.zip
100% 16.6k/16.6k [00:00<00:00, 14.0MB/s]


In [ ]:
# !unzip assignment_1_data.zip

Archive:  assignment_1_data.zip
   creating: assignment_1_data/
  inflating: assignment_1_data/input.json  
  inflating: assignment_1_data/output.json  


# Importing input data

In [2]:
import json
import re

In [3]:
with open('assignment_1_data/input.json','r', encoding='utf-8') as input_file:
  input_data = json.load(input_file)
  input_file.close()

In [4]:
with open('assignment_1_data/output.json','r', encoding='utf-8') as output_file:
  output_data = json.load(output_file)
  output_file.close()

In [5]:
print(input_data[0])
print(output_data[0])

{'sid': 0, 'input_tokens': ['For', 'every', '100', 'females', 'there', 'were', '90.0', 'males', '.']}
{'sid': 0, 'output_tokens': ['<self>', '<self>', 'one hundred', '<self>', '<self>', '<self>', 'ninety point zero', '<self>', 'sil']}


# Sample functions for generating the output

- If there is a numeric inside the string, the check for the following:
    - If there is no other character except for numerics, then it should be handled like a number.
    - If there is a symbol in front of numerics like symbols of currency, then they should be handled as per currency.
    - If there are any characters, then the string can either be a date or might be having some units associated with the numeric, eg: 5m.

In [6]:
# # Return the text version associated to the numeric in the input string
# def convert_number(input_string):
    

In [7]:
def is_symbol(input_string):
    if bool(re.search(r'\$', input_string)):
        return 'dollar'
    elif bool(re.search(r'\£', input_string)):
        return 'pound'
    elif bool(re.search(r'\€', input_string)):
        return 'euro'

In [8]:
is_symbol('€349')

'euro'

- If there are two 

In [9]:
# Returns if a punctuation is present in the input string
def has_punc(input_string):
    return bool(re.search(r'[^\w\s]', input_string))

In [10]:
# Returns true if the input string has a numeric present
def has_numeric(input_string):
    return bool(re.search(r'\d', input_string))

In [55]:
# Returns true if the input string has only 4 numerical characters, which would be indicative of an year
def has_fournumeric(input_string):
    return bool(re.search(r'^\d{4}$', input_string))

In [ ]:
def get_fournumeric(input_string):
    if(input_string[1] == )

In [59]:
has_fournumeric('45211')

False

In [11]:
def has_allcaps(input_string):    
    return bool(re.search(r'[A-Z]+$', input_string))

In [12]:
# Converst the abbreviated version to a spaced version as per output format
def get_allcaps(input_string):
    return ((' '.join(input_string)).lower() + ' ')

In [13]:
has_punc('/')

True

In [64]:
roman_numerals = {
    'I': 'first',
    'II': 'second',
    'III': 'third',
    'IV': 'fourth',
    'V': 'fifth',
    'VI': 'sixth',
    'VII': 'seventh',
    'VIII': 'eighth',
    'IX': 'ninth',
    'X': 'tenth',
    'XI': 'eleventh',
    'XII': 'twelfth',
}

In [14]:
full_form = {
        'm': 'meter', 
        'mm': 'millimeter',
        'cm': 'centimeter',
        'km': 'kilometer',

        'm2': 'square meter',
        'mm2': 'square millimeter',
        'cm2': 'square centimeter',
        'km2': 'square kilometer',
        'sq': 'square',

        'm3': 'cubic meter',
        'mm3': 'cubic millimeter',
        'cm3': 'cubic centimeter',
        'km3': 'cubic kilometer',

        'a.m.': 'a m',
        'p.m.': 'p m',

        'M': 'million',
        'm': 'million',
        'mn': 'million'

        'B': 'billion',
        'b': 'billion',
        'bn': 'billion'
        }

In [15]:
# Returns true if the input string forms a part of an abbreviation
def get_abbr(input_string):
    if input_string in full_form.keys():
        return full_form[input_string]

In [16]:
# bool(re.search(r'[A-Z]\.', input_string)

In [17]:
get_abbr('sq')

'square'

In [18]:
has_numeric('He has #22# dogs')

True

In [19]:
has_allcaps('BABA')

True

In [72]:
#todo: write your own solution.
def solution(input_tokens):
  sol = []
  flag = False

  for token in input_tokens:
    # Handling Punctuation.
    if((has_punc(token) and (len(token) == 1)) and not flag):
      sol.append('sil')
      flag = True
    
    # Handling roman numerals.
    if token in roman_numerals.keys() and not flag:
      sol.append('the ' + roman_numerals[token])
      flag = True

    # Handling all capital tokens (not as abbreviations though).
    if has_allcaps(token) and (len(token) > 1) and not flag:
      sol.append(get_allcaps(token))
      flag = True
    


    # If nothing else, then the token should be <self>
    if not flag:
      sol.append('<self>')

    flag = False    # preparing for the next iteration.

  return sol 


In [73]:
get_allcaps('ISBNf')

'i s b n f '

In [74]:
def solution_dump(solution_file_path):
  solution_data = []
  for input_sentence in input_data:
    solution_sid = input_sentence['sid']
    solution_tokens = solution(input_sentence['input_tokens'])
    solution_data.append({'sid':solution_sid,
                          'output_tokens':solution_tokens})

  with open(solution_file_path,'w') as solution_file:
    json.dump(solution_data, solution_file, indent=2, ensure_ascii=False)
    solution_file.close()

In [75]:
solution_dump('solution.json')

## Error in Solution wrt Ground Truth Data

In [76]:
with open('solution.json','r', encoding='utf-8') as solution_file:
  solution_data = json.load(solution_file)
  solution_file.close()

In [77]:
# Finding count of differences
error_list = {}
count = 0
percentage_error = 0
total_perc_error = 0

for output_sentence in output_data:
    res_sid = output_sentence['sid']
    res_token = output_sentence['output_tokens']

    sol_token  = solution_data[res_sid]['output_tokens']
    for i in range(len(res_token)):
        if (res_token[i] != sol_token[i]):
            count += 1
    percentage_error = 100*count/len(res_token)
    total_perc_error += percentage_error
    error_list[res_sid] = percentage_error
    count = 0

error_list['total'] = total_perc_error/len(error_list)

In [78]:
with open('error_list.json','w') as error_file:
    json.dump(error_list, error_file, indent=2, ensure_ascii=False)
    error_file.close()

# Expected submission

In [ ]:
'''
We expect a .py file which takes as input the input file path and the solution file path:

python run_assignment1.py --input_path <path_to_input> --solution_path <path_to_solution>

Once the solution file is generated, we will run a checker script to get the performance metrics:

python run_checker.py --solution_path <path_to_solution> --ground_truth_path <path_to_ground_truth>

'''

# Running on HPC

In [ ]:
'''
Students may use the hpc run their code. Please use the following commands to create a conda environment:


module load apps/anaconda/3EnvCreation
conda create --prefix=~/<environment_name> python=3.6
module unload apps/anaconda/3EnvCreation

This is a one-time process. Once the environment is created, the students may wish to install some packages. This can be done as follows:

module load apps/anaconda/3
conda activate ~/<environment_name>
module unload apps/anaconda/3
pip install <package_name> 

It is highly recommended that the students do not install PyTorch in this fashion. The recommended way of configuring PyTorch will be discused in the PyTorch tutorial class scheduled for later this semester.

This assignment will be entirely run on CPU. An interactive CPU job can be run as follows:

qsub -I -P <project_name> -l select=1:ncpus=1:mem=16G -l walltime=<hh:mm:ss>

Once the job is ready, the conda environment would have to be activated again as follows:

module load apps/anaconda/3
conda activate ~/<environment_name>
module unload apps/anaconda/3

Students may be interested to run a jupyter notebook on HPC. Kindly follow the steps from the link below:

https://supercomputing.iitd.ac.in/?softevents=jupyternotebook

'''